참고 : 내 로컬에는 tensorflow-datasets이 깔려 있다. aiffel 환경에 있음

In [ ]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.4.0


### Load data

In [ ]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteB1EIHN/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteB1EIHN/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteB1EIHN/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


### 데이터 분리하기
1. train, test 분리
2. 각 데이터셋의 sentence와 label 추출
3. numpy array로 바꿔주기

In [ ]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


### Tokenizing
1. Tokenizer instantiate
2. fit_on_texts
3. work_index : word index를 갖는 dictionary 생성
4. texts_to_sequences : 각 문장의 벡터값들을 하나의 sequence로 합치기
5. pad_sequences : 문장마다 길이가 다르므로 길이를 맞춰주기 위해서 0을 채움.

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


이렇게 생성된 word index는 {단어: 토큰}의 딕셔너리 형태이다. 이를 토큰: 단어로 바꿔줘야 우리는 토큰(벡터)를 이용하여 단어를 표현할 수 있음.

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3])) #decoded review
print(training_sentences[3]) #original review

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


*descend*의 경우 OOV로 나왔는데, 자주 등장하는 단어가 아니라서(자주 사용된 상위 10000개의 단어(num_words=10000)에 포함되지 않아서) 학습시 제외되었기 때문에 OOV로 처리되었다.

### Modeling
Embedding layer 추가.
* Flatten() or GlobalAveragePooling1D()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 7s 6ms/step - loss: 0.6083 - accuracy: 0.6327 - val_loss: 0.3486 - val_accuracy: 0.8445
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2357 - accuracy: 0.9142 - val_loss: 0.3755 - val_accuracy: 0.8352
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0918 - accuracy: 0.9804 - val_loss: 0.4499 - val_accuracy: 0.8303
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0212 - accuracy: 0.9977 - val_loss: 0.5223 - val_accuracy: 0.8265
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0068 - accuracy: 0.9994 - val_loss: 0.5904 - val_accuracy: 0.8270
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0022 - accuracy: 0.9999 - val_loss: 0.6334 - val_accuracy: 0.8292
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 8.8135e-04 - accuracy: 1.0000 - val_loss: 0.6768 - val_accuracy: 0.8290
Ep

### Visualization

In [ ]:
e = model.layers[0] # 첫 번째 레이어는 임베딩 레이어
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

이제 다운받은 vecs.tsv 파일이랑 meta.tsv 파일을 가지고 http://projector.tensorflow.org/ 에 접속한다. **Load** 버튼을 누르고 **Step 1**에 vecs.tsv 파일을 업로드, **Step 2**에 meta.tsv 파일을 업로드하면 프로젝터로 볼 수 있다!!

In [ ]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
